In [3]:
cd "/Users/TaihuaLi/Desktop/Projects/Netflix Prize/"

/Users/TaihuaLi/Desktop/Projects/Netflix Prize


In [4]:
import numpy as np
import pandas as pd

In [5]:
ratings = pd.read_csv("Rating Zeros.csv", index_col=0) # row: users  col: movies
items = pd.read_csv("items.csv", index_col=0) # movies information
genre = pd.read_csv("genre.csv") # movie genre information

In [6]:
ratings.head(3)

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5,3,4,3,3,5,4,1,5,3,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
items.head(3)

,Name,Unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantacy,Film.Noir,Horror,Musical,Mystery,Romance,Sci.Fi,Thriller,War,Western
﻿Movie.ID,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [8]:
genre.head(3)

,﻿Genre,ID
0,unknown,0
1,Action,1
2,Adventure,2


### K-means Clustering: Users

In [9]:
from sklearn.cluster import KMeans

In [10]:
kmeans = KMeans(n_clusters=5)

In [11]:
kmeans.fit(ratings,)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=5, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [12]:
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

In [13]:
pd.set_option('display.max_rows', ratings.describe().shape[1])
pd.DataFrame(centroids)

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.521164,0.023810,0.013228,0.060847,0.105820,0.068783,0.447090,0.208995,0.650794,0.166667,...,2.341877e-17,-1.214306e-17,2.341877e-17,-6.071532e-18,2.341877e-17,2.645503e-03,7.936508e-03,5.291005e-03,2.341877e-17,2.341877e-17
1,2.909605,0.033898,0.423729,0.141243,0.101695,0.101695,2.333333,0.175141,1.553672,0.361582,...,-1.301043e-18,-1.040834e-17,1.694915e-02,1.129944e-02,-1.301043e-18,-2.602085e-18,-1.301043e-18,-5.204170e-18,-1.301043e-18,-1.301043e-18
2,2.322034,0.711864,0.559322,2.661017,0.440678,0.559322,2.779661,2.898305,2.830508,1.644068,...,-2.602085e-18,6.779661e-02,-2.602085e-18,-3.035766e-18,-2.602085e-18,-1.517883e-18,-2.602085e-18,-3.035766e-18,-2.602085e-18,-2.602085e-18
3,2.360515,0.609442,0.231760,1.197425,0.257511,0.047210,1.884120,1.609442,1.175966,0.283262,...,1.287554e-02,-1.127570e-17,9.974660e-18,-5.637851e-18,1.287554e-02,-2.818926e-18,9.974660e-18,-5.637851e-18,9.974660e-18,9.974660e-18
4,3.687500,2.302083,1.104167,2.687500,1.458333,0.052083,3.166667,2.281250,2.114583,0.531250,...,-5.204170e-18,-8.673617e-18,-5.204170e-18,-4.336809e-18,-5.204170e-18,-2.168404e-18,-5.204170e-18,-4.336809e-18,3.125000e-02,3.125000e-02


In [14]:
movie_names = items.Name

In [15]:
genre_types = items.columns[1:]

In [16]:
items_array = np.array(items)
movie_genre = np.array(items[genre_types])

In [17]:
items_array

array([['Toy Story (1995)', 0, 0, ..., 0, 0, 0],
       ['GoldenEye (1995)', 0, 1, ..., 1, 0, 0],
       ['Four Rooms (1995)', 0, 0, ..., 1, 0, 0],
       ..., 
       ['Sliding Doors (1998)', 0, 0, ..., 0, 0, 0],
       ['You So Crazy (1994)', 0, 0, ..., 0, 0, 0],
       ['Scream of Stone (Schrei aus Stein) (1991)', 0, 0, ..., 0, 0, 0]], dtype=object)

##### Cluster Definition

In [18]:
def FiveStarMovies(centroids, labels, k, NTerms):
    five_star_movies = []
    cluster_movies = []
    for i in range(k):
        cluster_members = ratings[labels==i]
        print 'cluster',i+1,'size: ', cluster_members.shape[0]
        movie_total_rating = np.array(cluster_members.sum(0)).astype(float)
        movie_rating_count = np.array((cluster_members!=0).sum(0)).astype(float)
        movie_average_rating = np.divide(movie_total_rating, movie_rating_count)
        movie_average_rating[np.isnan(movie_average_rating)] = 0
        rated_movie_list = sorted(zip(movie_names, movie_average_rating), key=lambda x: int(x[1]), reverse=True)
        to_append = []
        cluster_append = []
        for j in rated_movie_list:
            if j[1] == 5:
                to_append.append(j[0])   
            if j[1] != 0:
                cluster_append.append(j[0])
        five_star_movies.append(to_append)
        cluster_movies.append(cluster_append)
    print "\n"
    # most rated movies in each cluster
    k = 0
    for m in five_star_movies:
        prototype = list(np.zeros(len(genre_types)))
        for n in items_array:    
            for i in m:
                if i == n[0]:
                    prototype += n[1:]
        print "Representative Movies:"
        for i in range(5):
            print m[i]
        print "\n"
        prototype = map(int, prototype)
        movie_genre_to_use = sorted(zip(genre_types, prototype), key=lambda x:int(x[1]), reverse=True)
        print "Cluster", k+1, "top genre: "
        for i in movie_genre_to_use[:NTerms]:
            print i[0]
        print "\n"
        k += 1
    return five_star_movies, cluster_movies

In [19]:
five_star_movies, cluster_movies = FiveStarMovies(centroids, labels, 5, 5)

cluster 1 size:  378
cluster 2 size:  177
cluster 3 size:  59
cluster 4 size:  233
cluster 5 size:  96


Representative Movies:
Four Rooms (1995)
Faster Pussycat! Kill! Kill! (1965)
Aristocats, The (1970)
Homeward Bound: The Incredible Journey (1993)
Paradise Lost: The Child Murders at Robin Hood Hills (1996


Cluster 1 top genre: 
Drama
Comedy
Children
Adventure
Romance


Representative Movies:
Dolores Claiborne (1994)
Three Colors: Red (1994)
So I Married an Axe Murderer (1993)
Gone with the Wind (1939)
Glengarry Glen Ross (1992)


Cluster 2 top genre: 
Drama
Comedy
Romance
Thriller
Action


Representative Movies:
Stalingrad (1993)
Baby-Sitters Club, The (1995)
Timecop (1994)
Great Day in Harlem, A (1994)
Two or Three Things I Know About Her (1966)


Cluster 3 top genre: 
Drama
Comedy
Animation
Documentary
Romance


Representative Movies:
Maya Lin: A Strong Clear Vision (1994)
Turbo: A Power Rangers Movie (1997)
Pillow Book, The (1995)
Perfect Candidate, A (1996)
Washington Square (1